In [11]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.model_selection import train_test_split

# ========================================================
# ⚙️ CẤU HÌNH 
# ========================================================
# 1. Nguồn Labels 
LABEL_DIR = "/kaggle/input/go-cafa6"
# 2. Nguồn Embeddings
EMBED_DIR = "/kaggle/input/cafa6-embeds"
TRAIN_IDS_FILE = os.path.join(EMBED_DIR, "train_ids.txt")
# 3. Chia tập
VAL_SIZE = 0.1  # 10% cho Validation
SEED = 42

# =========================================================
# HÀM XỬ LÝ CHÍNH: TẠO TARGETS & SPLIT (CHẠY 3 LẦN)
# =========================================================

def process_and_split(coverage_name):
    """
    Thực hiện toàn bộ pipeline từ Load Vocab đến Split cho một mức coverage cụ thể.
    coverage_name là 'C90', 'C95', hoặc 'C99'.
    """
    # 1. DYNAMIC FILE PATHS (Dựa trên tên coverage_name và chiến lược '_remove')
    TRAIN_LABEL_FILE = os.path.join(LABEL_DIR, f"train_data_{coverage_name}_remove.tsv")
    VOCAB_FILE = os.path.join(LABEL_DIR, f"vocab_{coverage_name}_remove.csv")
    
    # 2. OUTPUT FILES
    OUTPUT_PKL = f"train_targets_{coverage_name}.pkl"
    TRAIN_OUT_IDS = f"train_ids_{coverage_name}_split.npy"
    VAL_OUT_IDS = f"val_ids_{coverage_name}_split.npy"

    print(f"\n========================================================")
    print(f"🚀 PROCESSING SET: {coverage_name} ({VAL_SIZE*100:.0f}% Validation)")
    print(f"========================================================")

    # BƯỚC 1: LOAD VOCAB & TẠO MAP (TERM -> INDEX)
    print(f"1. Loading Vocab từ {VOCAB_FILE}...")
    vocab_df = pd.read_csv(VOCAB_FILE)
    term_to_idx = {term: i for i, term in enumerate(vocab_df['term'])}
    num_classes = len(term_to_idx)
    print(f"   - Số lượng nhãn (Classes): {num_classes}")

    # BƯỚC 2: LOAD EMBEDDING PROTEIN IDs (FILTER INPUT)
    print(f"\n2. Loading Embedding IDs từ {TRAIN_IDS_FILE}...")
    if TRAIN_IDS_FILE.endswith('.npy'):
        embed_protein_ids = np.load(TRAIN_IDS_FILE)
    else:
        with open(TRAIN_IDS_FILE, 'r') as f:
            embed_protein_ids = [line.strip() for line in f if line.strip()]
    
    valid_proteins_set = set(embed_protein_ids)
    print(f"   - Số lượng Protein có Embeddings: {len(valid_proteins_set):,}")

    # BƯỚC 3: LOAD LABELS & FILTER & MAP
    print(f"\n3. Loading & Processing Labels từ {TRAIN_LABEL_FILE}...")
    df = pd.read_csv(TRAIN_LABEL_FILE, sep="\t", names=["protein", "term", "aspect"])
    
    df = df[df['protein'].isin(valid_proteins_set)]
    df = df[df['term'].isin(term_to_idx)]
    
    print(f"   - Số proteins unique hợp lệ: {df['protein'].nunique():,}")

    # Chuyển đổi: GO Term -> Index (Số nguyên)
    df['label_idx'] = df['term'].map(term_to_idx)

    # BƯỚC 4: TẠO MULTI-HOT DICTIONARY
    print("\n4. Grouping by Protein (Creating Dictionary)...")
    labels_dict = df.groupby('protein')['label_idx'].apply(list).to_dict()

    with open(OUTPUT_PKL, 'wb') as f:
        pickle.dump(labels_dict, f)
    print(f"   ✅ Đã lưu targets: {OUTPUT_PKL}")

    # BƯỚC 5: CHIA TRAIN / VALIDATION (SPLIT)
    print(f"\n5. Splitting Train/Val...")
    all_proteins = np.array(list(labels_dict.keys()))

    train_ids, val_ids = train_test_split(
        all_proteins, 
        test_size=VAL_SIZE, 
        random_state=SEED,
        shuffle=True
    )

    # Lưu danh sách ID ra file .npy
    np.save(TRAIN_OUT_IDS, train_ids)
    np.save(VAL_OUT_IDS, val_ids)

    print(f"   ✅ Đã lưu file IDs: {TRAIN_OUT_IDS} và {VAL_OUT_IDS}")
    print(f"   - Train Samples: {len(train_ids):,} | Val Samples: {len(val_ids):,}")


# =========================================================
# THỰC THI CHƯƠNG TRÌNH CHÍNH (TẠO 3 BỘ)
# =========================================================
print("🚀 BẮT ĐẦU TẠO 3 BỘ DỮ LIỆU ĐẦU RA...")

for name in ['C90', 'C95', 'C99']:
    process_and_split(name)

print("\n" + "="*50)
print("✅ HOÀN TẤT! ĐÃ TẠO THÀNH CÔNG 3 BỘ DỮ LIỆU ĐẦU RA.")

🚀 BẮT ĐẦU TẠO 3 BỘ DỮ LIỆU ĐẦU RA...

🚀 PROCESSING SET: C90 (10% Validation)
1. Loading Vocab từ /kaggle/input/go-cafa6/vocab_C90_remove.csv...
   - Số lượng nhãn (Classes): 3375

2. Loading Embedding IDs từ /kaggle/input/cafa6-embeds/train_ids.txt...
   - Số lượng Protein có Embeddings: 82,404

3. Loading & Processing Labels từ /kaggle/input/go-cafa6/train_data_C90_remove.tsv...
   - Số proteins unique hợp lệ: 82,401

4. Grouping by Protein (Creating Dictionary)...
   ✅ Đã lưu targets: train_targets_C90.pkl

5. Splitting Train/Val...
   ✅ Đã lưu file IDs: train_ids_C90_split.npy và val_ids_C90_split.npy
   - Train Samples: 74,160 | Val Samples: 8,241

🚀 PROCESSING SET: C95 (10% Validation)
1. Loading Vocab từ /kaggle/input/go-cafa6/vocab_C95_remove.csv...
   - Số lượng nhãn (Classes): 6413

2. Loading Embedding IDs từ /kaggle/input/cafa6-embeds/train_ids.txt...
   - Số lượng Protein có Embeddings: 82,404

3. Loading & Processing Labels từ /kaggle/input/go-cafa6/train_data_C95_remove.t

In [10]:
import pickle
import numpy as np
import pandas as pd
import os

# Cấu hình đường dẫn
TARGET_PKL = "/kaggle/working/train_targets_C95.pkl"

# 1. Load targets dictionary
print(f"Đang đọc file targets: {TARGET_PKL}...")
with open(TARGET_PKL, 'rb') as f:
    labels_dict = pickle.load(f)

# 2. Load Vocab (để dịch index)
print(f"Đang đọc file vocab: {VOCAB_FILE}...")
vocab_df = pd.read_csv(VOCAB_FILE)
term_list = vocab_df['term'].tolist()
aspect_list = vocab_df['aspect'].tolist()

# 3. Lấy mẫu và kiểm tra
sample_id = list(labels_dict.keys())[0]
sample_indices = labels_dict[sample_id]
num_classes = len(term_list)

print("\n--- KIỂM TRA MẪU ---")
print(f"Protein ID: {sample_id}")
print(f"Số lượng nhãn dương: {len(sample_indices)}")
print(f"Total classes: {num_classes}")

# 4. Dịch các chỉ số thành GO Term và Aspect
print("\nDanh sách nhãn GO (dịch từ Index):")
for idx in sample_indices:
    term = term_list[idx]
    aspect = aspect_list[idx]
    print(f"  - Index {idx:<5} -> {term:<12} (Aspect: {aspect})")

# 5. Kiểm tra dạng Multi-hot (chuyển đổi)
# Tạo vector 0/1 dài 6416 số
multi_hot_vector = np.zeros(num_classes, dtype=np.int8)
multi_hot_vector[sample_indices] = 1

print(f"\nVector Multi-hot (ví dụ): {multi_hot_vector[:10]}...")

multi_hot_vector.shape

Đang đọc file targets: /kaggle/working/train_targets_C95.pkl...
Đang đọc file vocab: /kaggle/input/go-cafa6/vocab_C95_remove.csv...

--- KIỂM TRA MẪU ---
Protein ID: A0A023FBW4
Số lượng nhãn dương: 4
Total classes: 6413

Danh sách nhãn GO (dịch từ Index):
  - Index 3     -> GO:0005488   (Aspect: MFO)
  - Index 4449  -> GO:0019956   (Aspect: MFO)
  - Index 2121  -> GO:0019955   (Aspect: MFO)
  - Index 6     -> GO:0005515   (Aspect: MFO)

Vector Multi-hot (ví dụ): [0 0 0 1 0 0 1 0 0 0]...


(6413,)